In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import streamlit as st
import plotly.express as px
import plotly.graph_objects as go
import datetime as dt
from haversine import haversine 
import re
import numpy as np

In [2]:
df = pd.read_csv('C:/Users/carlo/Desktop/Python_piva/Curry company/train2.csv')

In [3]:
df1 = df.copy()

In [5]:
# Removendo colunas que contém 'NaN'

coluna = ['Delivery_person_Age', 'Road_traffic_density', 
		  'Festival', 'City', 'multiple_deliveries', 
    	  'Delivery_person_Ratings', 'Time_taken(min)']

for col in coluna:
    linhas_selecionadas = df1[col] != 'NaN '
    df1 = df1.loc[linhas_selecionadas, :].copy()
    

In [6]:
# Convertendo as colunas para o tipo correto

df1['Delivery_person_Age'] = df1['Delivery_person_Age'].astype(int)

df1['Delivery_person_Ratings'] = df1['Delivery_person_Ratings'].astype(float)

df1['Order_Date'] = pd.to_datetime(df1['Order_Date'], format='%d-%m-%Y')

df1['Time_Orderd'] = pd.to_datetime(df1['Time_Orderd'])

df1['Time_Order_picked'] = pd.to_datetime(df1['Time_Order_picked'])

df1['multiple_deliveries'] = df1['multiple_deliveries'].astype(int)

#df1['Time_taken(min)'] = df1['Time_taken(min)'].astype(int)

C:\Users\carlo\AppData\Local\Temp\ipykernel_6740\2051886209.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df1['Time_Orderd'] = pd.to_datetime(df1['Time_Orderd'])
C:\Users\carlo\AppData\Local\Temp\ipykernel_6740\2051886209.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df1['Time_Order_picked'] = pd.to_datetime(df1['Time_Order_picked'])


In [7]:
# Limpando a coluna 'Time_taken(min)'
# Assumindo que seu DataFrame se chama df1
df1['Time_taken_numeric'] = df1['Time_taken(min)'].apply(lambda x: re.findall(r'\d+', str(x))[0])

# Convertendo para inteiro
df1['Time_taken_numeric'] = df1['Time_taken_numeric'].astype(int)

In [ ]:
df1.head()

In [8]:
# Entregas unicas
entregas_unicas = len(df1.loc[:, 'Delivery_person_ID'].unique())

entregas_unicas

1320

In [9]:
# Distância média dos restaurantes e dos locais de entrega
col = ['Restaurant_latitude', 'Restaurant_longitude', 'Delivery_location_latitude', 'Delivery_location_longitude']

df1['Distance'] = df1[col].apply(lambda x: haversine(
    (x['Restaurant_latitude'], x['Restaurant_longitude']),
    (x['Delivery_location_latitude'], x['Delivery_location_longitude'])
), axis=1)

media_distancia = df1['Distance'].mean()

print('A distância média das entregas é: {:.2f}'.format(media_distancia))


A distância média das entregas é: 26.91


In [ ]:
# Distância média dos restaurantes e dos locais de entrega
col = ['Restaurant_latitude','Restaurant_longitude', 'Delivery_location_latitude', 'Delivery_location_longitude']

media_distancia =df1['Distance'] = df1[col].apply(lambda x: haversine( 
    (x['Restaurant_latitude'],x['Restaurant_longitude']),
    (x['Delivery_location_latitude'], x['Delivery_location_longitude']), 
	), axis=1)
media_distancia = df1.loc[:, ['City', 'Distance']].groupby('City').mean().reset_index()

fig = go.Figure(data=[go.Pie(labels=media_distancia['City'], values=media_distancia['Distance'], pull=[0.02, 0.1, 0.02], title='Distância Média entre Restaurantes e Local de Entrega')])
fig.show()

In [ ]:
# Média de entregas por Cidade
col = ['Time_taken_numeric', 'City']
media_entrega_cidade = df1.loc[:, col].groupby('City').agg({'Time_taken_numeric': ['mean', 'std']})

media_entrega_cidade.columns =['Media_tempo', 'Desvio_tempo']

media_entrega = media_entrega_cidade.reset_index()

media_entrega

In [ ]:
col = ['Time_taken_numeric', 'City']

media_entrega_cidade= df1.loc[:, col].groupby('City').agg({'Time_taken_numeric': ['mean', 'std']})

fig = px.box(df1, x='City', y='Time_taken_numeric', title='Distribuição de Tempo de Entrega por Cidade')
fig.show()

In [ ]:
# média de entregas por tipo de pedido
col = ['Time_taken_numeric', 'City', 'Type_of_order']
media_entrega_cidade = df1.loc[:, col].groupby(['City', 'Type_of_order']).agg({'Time_taken_numeric': ['mean', 'std']})

media_entrega_cidade.columns =['Media_tempo', 'Desvio_tempo']

media_entrega = media_entrega_cidade.reset_index()

media_entrega

In [ ]:
#Média de entregas por tipo de tráfego
col = ['Time_taken_numeric', 'City', 'Road_traffic_density']

media_entrega_cidade = df1.loc[:, col].groupby(['City', 'Road_traffic_density']).agg({'Time_taken_numeric': ['mean', 'std']})

media_entrega_cidade.columns = ['Media_tempo', 'Desvio_tempo']

media_entrega = media_entrega_cidade.reset_index()

media_entrega

In [ ]:
#Média de entregas por tipo de tráfego
col = ['Time_taken_numeric', 'City', 'Road_traffic_density']
media_entrega_cidade = df1.loc[:, col].groupby(['City', 'Road_traffic_density']).agg({'Time_taken_numeric': ['mean', 'std']})

media_entrega_cidade.columns = ['Media_tempo', 'Desvio_tempo']

media_entrega = media_entrega_cidade.reset_index()

fig = px.sunburst(media_entrega, path=['Road_traffic_density', 'City'], values='Media_tempo', color_continuous_scale='RdBu')           

fig.show()


In [ ]:
# Entregas por festival
col = ['Time_taken_numeric', 'Festival']
media_entrega_festival = df1.loc[:, col].groupby('Festival').agg({'Time_taken_numeric': ['mean', 'std']})

media_entrega_festival.columns = ['Media_tempo', 'Desvio_tempo']

media_entrega = media_entrega_festival.reset_index()

linhas_selecionadas = media_entrega['Festival'] == 'Yes '
media_entrega =media_entrega.loc[linhas_selecionadas,: ]

media_entrega

